In [28]:
import json
import os, sys

where_is_the_data = os.path.abspath(os.path.join(os.path.dirname("."), '../..', 'ingest/test/data'))
sys.path.append(where_is_the_data)

# load json data from file mock_coinmarketcap_data.json
with open(f'{where_is_the_data}/mock_coinmarketcap_data.json', 'r') as f:
    mock_coinmarketcap_data = json.load(f)


In [40]:
coin_schema = {
            'id': 'int64',
            'name': 'object',
            'symbol': 'object',
            'slug': 'object',
            'num_market_pairs': 'int64',
            'date_added': 'object',
            'price': 'float64',
            'fiat': 'object',
            'market_cap': 'float64',
            'volume_24h': 'float64',
            'volume_change_24h': 'float64',
            'percent_change_1h': 'float64',
            'percent_change_24h': 'float64',
            'percent_change_7d': 'float64',
            "percent_change_30d": 'float64',
            "percent_change_60d": 'float64',
            "percent_change_90d": 'float64',
            "market_cap": 'float64',
            "market_cap_dominance": 'float64',
            "fully_diluted_market_cap": 'float64',
            "tvl": 'float64',
            'tags': 'int64',
            "max_supply": 'float64',
            "circulating_supply": 'int64',
            "total_supply": 'int64',
            "is_active": 'bool',
            "infinite_supply": 'bool',
            "platform": 'object',
            "cmc_rank": 'int64',
            "is_fiat": 'bool',
            "self_reported_circulating_supply": 'object',
            "self_reported_market_cap": 'object',
            "tvl_ratio": 'float64',
            "last_updated": 'object',
        }

In [51]:
flat_data = []

for key, obj in mock_coinmarketcap_data['data'].items():
    flat_coin_data = {}
    for k in obj.keys():
        if k not in coin_schema.keys():
            
            if k == 'quote':
                for quote in obj[k].keys():
                    flat_coin_data['fiat'] = quote
                for quote in obj[k].values():
                    for qk in quote.keys():
                        if qk in coin_schema.keys():
                            flat_coin_data[qk] = quote[qk]
        else:
            if k == 'tags':
                flat_coin_data[k] = len(obj[k])
            else:
                flat_coin_data[k] = mock_coinmarketcap_data['data'][key][k]
    flat_data.append(flat_coin_data)

In [61]:
# crate pandas dataframe from flat_data with schema from coin_schema
import pandas as pd
df = pd.DataFrame(flat_data, columns=coin_schema.keys())
df

,id,name,symbol,slug,num_market_pairs,date_added,price,fiat,market_cap,volume_24h,...,total_supply,is_active,infinite_supply,platform,cmc_rank,is_fiat,self_reported_circulating_supply,self_reported_market_cap,tvl_ratio,last_updated
0,1,Bitcoin,BTC,bitcoin,12285,2010-07-13T00: 00: 00.000Z,117726.543145,USD,2.343597e+12,6.371419e+10,...,1.990713e+07,1,0,None,1,0,NaN,NaN,None,2025-08-16T12: 34: 00.000Z
1,1027,Ethereum,ETH,ethereum,10441,2015-08-07T00: 00: 00.000Z,4406.778997,USD,5.319331e+11,4.962864e+10,...,1.207079e+08,1,1,None,2,0,NaN,NaN,None,2025-08-16T12: 34: 00.000Z
2,5426,Solana,SOL,solana,969,2020-04-10T00: 00: 00.000Z,186.420991,USD,1.006542e+11,6.777183e+09,...,6.076174e+08,1,1,None,6,0,525236893.3,9.791518e+10,None,2025-08-16T12: 34: 00.000Z


In [70]:
# have a better understanding of the data types
for key in df.dtypes.keys():
    if coin_schema[key] == df[key].dtype.name:
        pass
    else:
        print(f"Column '{key}' has type {df[key].dtype.name}, expected {coin_schema[key]}")   


In [78]:
# have a better understanding of the data types
for key in df.columns:
    
    print(f"Converting column '{key}' to type {coin_schema[key]}")
    df[key] = df[key].astype(coin_schema[key])

Converting column 'id' to type int64
Converting column 'name' to type object
Converting column 'symbol' to type object
Converting column 'slug' to type object
Converting column 'num_market_pairs' to type int64
Converting column 'date_added' to type object
Converting column 'price' to type float64
Converting column 'fiat' to type object
Converting column 'market_cap' to type float64
Converting column 'volume_24h' to type float64
Converting column 'volume_change_24h' to type float64
Converting column 'percent_change_1h' to type float64
Converting column 'percent_change_24h' to type float64
Converting column 'percent_change_7d' to type float64
Converting column 'percent_change_30d' to type float64
Converting column 'percent_change_60d' to type float64
Converting column 'percent_change_90d' to type float64
Converting column 'market_cap_dominance' to type float64
Converting column 'fully_diluted_market_cap' to type float64
Converting column 'tvl' to type float64
Converting column 'tags' to 

In [85]:
# apply the schema to the dataframe
df = df.astype(coin_schema)

In [91]:
# load fiat mock_exchange data
with open(f'{where_is_the_data}/mock_exchange_data.json', 'r') as f:
    mock_exchange_data = json.load(f)

In [92]:
mock_exchange_data

{'success': True,
 'terms': 'https://currencylayer.com/terms',
 'privacy': 'https://currencylayer.com/privacy',
 'timestamp': 1755262145,
 'source': 'USD',
 'quotes': {'USDGBP': 0.737385,
  'USDCAD': 1.37905,
  'USDJPY': 146.913014,
  'USDMXN': 18.730601}}

In [88]:
from time import localtime, strftime

# prepare flat fiat data per currency
flat_fiat_data = {}
if mock_exchange_data['success']:
    source = mock_exchange_data['source']
    per_fiat_data = {}

    per_fiat_data['timestamp'] = strftime('%Y-%m-%d %H:%M:%S', localtime(mock_exchange_data['timestamp']))
    per_fiat_data['source'] = source
    for key, value in mock_exchange_data['quotes'].items():
        per_fiat_data[key[len(source):]] = value

    flat_fiat_data[source] = per_fiat_data

flat_fiat_data



{'USD': {'timestamp': '2025-08-15 13:49:05',
  'source': 'USD',
  'GBP': 0.737385,
  'CAD': 1.37905,
  'JPY': 146.913014,
  'MXN': 18.730601}}

In [89]:
# add fiat data to the dataframe
from decimal import Decimal


def add_fiat_data(row):
    source = row['fiat']
    price = row['price']
    fiat_data = flat_fiat_data.get(source, {})
    if fiat_data:
        row['fiat_timestamp'] = fiat_data.get('timestamp', '')
        for key, value in fiat_data.items():
            if key != 'timestamp' and key != 'source':
                row[key] = Decimal(price * value if isinstance(value, (int, float)) else value)
    return row


df.apply(add_fiat_data, axis=1)


,id,name,symbol,slug,num_market_pairs,date_added,price,fiat,market_cap,volume_24h,...,is_fiat,self_reported_circulating_supply,self_reported_market_cap,tvl_ratio,last_updated,fiat_timestamp,GBP,CAD,JPY,MXN
0,1,Bitcoin,BTC,bitcoin,12285,2010-07-13T00: 00: 00.000Z,117726.543145,USD,2.343597e+12,6.371419e+10,...,False,NaN,NaN,NaN,2025-08-16T12: 34: 00.000Z,2025-08-15 13:49:05,86809.787017310751252807676792144775390625,162350.78932473863824270665645599365234375,17295561.281299717724323272705078125,2205088.9067667876370251178741455078125
1,1027,Ethereum,ETH,ethereum,10441,2015-08-07T00: 00: 00.000Z,4406.778997,USD,5.319331e+11,4.962864e+10,...,False,NaN,NaN,NaN,2025-08-16T12: 34: 00.000Z,2025-08-15 13:49:05,3249.49273070262643159367144107818603515625,6077.168575812442213646136224269866943359375,647413.18448112346231937408447265625,82541.619087981656775809824466705322265625
2,5426,Solana,SOL,solana,969,2020-04-10T00: 00: 00.000Z,186.420991,USD,1.006542e+11,6.777183e+09,...,False,525236893.3,9.791518e+10,NaN,2025-08-16T12: 34: 00.000Z,2025-08-15 13:49:05,137.46404276699257707150536589324474334716796875,257.083868234126157403807155787944793701171875,27387.669724124818458221852779388427734375,3491.77720853485470797750167548656463623046875


In [90]:
df

,id,name,symbol,slug,num_market_pairs,date_added,price,fiat,market_cap,volume_24h,...,total_supply,is_active,infinite_supply,platform,cmc_rank,is_fiat,self_reported_circulating_supply,self_reported_market_cap,tvl_ratio,last_updated
0,1,Bitcoin,BTC,bitcoin,12285,2010-07-13T00: 00: 00.000Z,117726.543145,USD,2.343597e+12,6.371419e+10,...,19907128,True,False,None,1,False,NaN,NaN,NaN,2025-08-16T12: 34: 00.000Z
1,1027,Ethereum,ETH,ethereum,10441,2015-08-07T00: 00: 00.000Z,4406.778997,USD,5.319331e+11,4.962864e+10,...,120707912,True,True,None,2,False,NaN,NaN,NaN,2025-08-16T12: 34: 00.000Z
2,5426,Solana,SOL,solana,969,2020-04-10T00: 00: 00.000Z,186.420991,USD,1.006542e+11,6.777183e+09,...,607617403,True,True,None,6,False,525236893.3,97915182385.583542,NaN,2025-08-16T12: 34: 00.000Z
